In [45]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [46]:
#read clean dataset
data = pd.read_pickle("data/COR_MAD_all.pkl").reset_index(drop=True)

In [47]:
data.loc[301030, "meta"]

'{"Turista": {"Promo": {"price": 39.3, "seats": 256}, "Promo +": {"price": 42.5, "seats": 256}, "Flexible": {"price": 63.4, "seats": 256}}, "Preferente": {"Promo": {"price": 66.2, "seats": 68}, "Promo +": {"price": 71.55, "seats": 68}, "Flexible": {"price": 106.8, "seats": 68}}}'

In [48]:
# generate train_id
data["train_id"] = data[["origin", "destination", "departure","arrival"]].sum(axis=1)

In [49]:
data["weekday"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday()
)
data["depart_month"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month
)
data["depart_hour"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour
)
# only jan, feb, march,
data = data.loc[data["depart_month"] < 4].reset_index(drop=True)

In [27]:
data["depart_year"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").year
)

In [28]:
data["depart_year"].value_counts()

2020    309944
Name: depart_year, dtype: int64

In [29]:
len(data["train_id"].unique())

1409

In [30]:
# a price-seat adatok a meta változóban vannak,
# ami egy mindig más struktúrájú dictionary,
# elég trükkös volt kinyerni, arra írtam ezt a fgv-t


def extract_meta(train):
    prices_seats = pd.DataFrame()
    for i in tqdm(range(train.shape[0])):
        dic_in = json.loads(train.meta[i])
        # extract values from tree dictionary
        while type(list(dic_in.values())[0]) != float:
            res = {key: list(value) for key, value in dic_in.items()}
            dic_out = {}
            for key in res.keys():
                for value in res[key]:
                    dic_out[key + "_" + value] = dic_in[key][value]
            dic_in = dic_out

        # check for seats, if no seat its the price
        for key in list(dic_in.keys()):
            if "seats" not in key:
                if "price" not in key:
                    dic_in[key + "_price"] = dic_in[key]
                    dic_in.pop(key)

        prices_seats = pd.concat(
            [prices_seats, pd.DataFrame.from_dict(dic_in, orient="index").T]
        ).reset_index(drop=True)
    return prices_seats

In [31]:
data.shape

(309944, 19)

In [32]:
chunks = {
    i: pd.concat(
        [
            data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True),
            extract_meta(
                data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True)
            ),
        ],
        axis=1,
    )
    for i in range(30)
}

100%|██████████| 10000/10000 [00:41<00:00, 242.95it/s]


In [33]:
df = pd.concat(list(chunks.values()))

In [34]:
last_chunk = pd.concat(
    [
        data.loc[300000:, :].reset_index(drop=True),
        extract_meta(data.loc[300000:, :].reset_index(drop=True)),
    ]
)

100%|██████████| 9944/9944 [00:46<00:00, 211.83it/s]


In [35]:
df = pd.concat([df,last_chunk])

In [36]:
df = df.dropna(subset=["id"]).reset_index(drop=True)

In [37]:
df.columns

Index(['id', 'company', 'origin', 'destination', 'departure', 'arrival',
       'duration', 'vehicle_type', 'vehicle_class', 'price', 'fare', 'seats',
       'meta', 'insert_date', 'train_id', 'weekday', 'depart_month',
       'depart_hour', 'depart_year', 'Turista_Flexible_price',
       'Turista_Flexible_seats', 'Preferente_Flexible_price',
       'Preferente_Flexible_seats', 'Turista_Promo_price',
       'Turista_Promo_seats', 'Turista_Promo +_price', 'Turista_Promo +_seats',
       'Preferente_Promo_price', 'Preferente_Promo_seats',
       'Preferente_Promo +_price', 'Preferente_Promo +_seats',
       'Turista Plus_Flexible_price', 'Turista Plus_Flexible_seats',
       'Preferente_Mesa_price', 'Preferente_Mesa_seats', 'Turista_Mesa_price',
       'Turista_Mesa_seats', 'Turista Plus_Promo_price',
       'Turista Plus_Promo_seats', 'Turista Plus_Promo +_price',
       'Turista Plus_Promo +_seats', 'Turista_Grupos Ida_price',
       'Turista_Grupos Ida_seats', 'Preferente_Grupos Ida_p

In [38]:
df = df.drop(
    columns=[
        "Preferente_YOVOY_price",
        "Preferente_YOVOY_seats",
        "Turista_Grupos Ida_price",
        "Turista_Grupos Ida_seats",
        "Preferente_Grupos Ida_price",
        "Preferente_Grupos Ida_seats",
        # "Turista Plus_YOVOY_price",
        # "Turista Plus_YOVOY_seats",
        # "Turista_YOVOY VERANO_price",
        # "Turista_YOVOY VERANO_seats",
        # "Preferente_YOVOY VERANO_price",
        # "Preferente_YOVOY VERANO_seats",
        # "Turista Plus_YOVOY VERANO_price",
        # "Turista Plus_YOVOY VERANO_seats",
        # "Preferente_Mesa_seats",
        # "Turista_Mesa_seats",
        # "Turista Plus_Mesa_seats",
    ]
)

In [39]:
df["days_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    ).days
    for i in range(df.shape[0])
]

df["timedelta_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    )
    for i in range(df.shape[0])
]

In [40]:
df.to_pickle("data/COR_MAD_trains.pkl")

___

In [41]:
data_1 = pd.read_pickle("data/COR_MAD_trains.pkl")
data_2 = pd.read_pickle("data/MAD_COR_trains.pkl")

In [42]:
columns = [
    "origin",
    "destination",
    "departure",
    "arrival",
    "duration",
    "insert_date",
    "train_id",
    "weekday",
    "depart_month",
    "depart_hour",
    "depart_year",
    "Turista_Promo_price",
    "Turista_Promo +_price",
    "Preferente_Promo_price",
    "Preferente_Promo +_price",
    "Turista Plus_Promo_price",
    "Turista Plus_Promo +_price",
    "Turista_Promo_seats",
    "Turista_Promo +_seats",
    "Preferente_Promo_seats",
    "Preferente_Promo +_seats",
    "Turista Plus_Promo_seats",
    "Turista Plus_Promo +_seats",
    "days_till_dep",
    "timedelta_till_dep",
]

In [43]:
data_all = pd.concat([data_1[columns], data_2[columns]]).reset_index(drop=True)

In [44]:
data_all.to_pickle("data/COR_MAD_trains_clean.pkl")